In [1]:
import sys
import os
from os.path import join
sys.path.append(os.path.abspath(".."))
from src.utils.preprocessing import preprocess_pipeline
from os.path import join
from src.data.mnist_loader import MnistDataloader

input_path = "../data"

mnist = MnistDataloader(
    join(input_path, "train-images-idx3-ubyte/train-images-idx3-ubyte"),
    join(input_path, "train-labels-idx1-ubyte/train-labels-idx1-ubyte"),
    join(input_path, "t10k-images-idx3-ubyte/t10k-images-idx3-ubyte"),
    join(input_path, "t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte"),
)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Training data (with augmentation)
x_train_prep, y_train_prep = preprocess_pipeline(
    x_train, y_train, augment=True
)

# Test data (no augmentation)
x_test_prep, y_test_prep = preprocess_pipeline(
    x_test, y_test, augment=False
)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert test data to tensors if not already done
x_test_tensor = torch.tensor(x_test_prep, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_prep, dtype=torch.long)

# Create DataLoader
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 5.4 MB/s  0:00:27m0:00:0100:01


In [4]:
from src.models.cnn import MNISTCNN

model = MNISTCNN()
model.load_state_dict(torch.load("models/mnist_cnn.pth"))
model.eval()  # Important: set model to evaluation mode


MNISTCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
from src.evaluation.evaluation import evaluate_model
from src.models.cnn import MNISTCNN


In [ ]:
model = MNISTCNN()
model.load_state_dict(torch.load("models/mnist_cnn.pth"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = evaluate_model(model, test_loader, device)

print("Test accuracy:", results["accuracy"])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm = results["confusion_matrix"]

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import numpy as np

preds = results["predictions"]
labels = results["labels"]

mis_idx = np.where(preds != labels)[0]


In [ ]:
# Visualize misclassified images
plt.figure(figsize=(12, 6))
for i, idx in enumerate(mis_idx[:10]):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_test_prep[idx].squeeze(), cmap="gray")
    plt.title(f"T:{labels[idx]} P:{preds[idx]}")
    plt.axis("off")
plt.show()
